<a href="https://colab.research.google.com/github/NP04-probability/DKReadPub/blob/master/DKRead.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## To do
* Reorganise sections (move parameters to end)
* Prior function

# Setup

## Importing libraries

In [0]:
import sys
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

# Plot setup

## Generate plot
Plots data and yfun on a panel with a linear and a logarithmic scale. Puts error bars on the data assuming sqrt(n).

It applies the necessary care with negative numbers on the log plot.

After calling, call plt.show(). (Allows setup of several panels)

In [0]:
def doDataPlot(nbins0,data,yfun, saveFigs=False):
    """
    data = np-array of the data points
      yfun = np-array of the fitted function values
      nbins = #bins (I have removed this (nbins0 is not used), as we can find it from the length of data or yfun).
    """

    nbins, = data.shape          # Get the number of bins from the data array, yfun should be the same
    x = np.arange(nbins)         # 1-D array 0,1,2...nbins-1
    xerr = 1 + np.zeros(nbins)   # 1-D array size nbins, all have value 1
    yerrH = np.sqrt(data)        # 1-D array size nbins
    yerrL = data - np.maximum(1e-2,data - yerrH)  # fix non-negative for logy

    ## yf = np.exp(-x)*0.8       # Something to plot as a line that is a bit different from the data

    fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True)  # Two plots on panel
    fig.patch.set_facecolor('w')

    ax = axs[0]                  # Commands to ax affect the top plot
    ## ax.set_yscale('log')
    ax.errorbar(x, data, yerr=[yerrL,yerrH], fmt='--o')  # Plot the data
    ax.plot(x,yfun,color='r')    # Plot the function
    ax.set_title('Giles playing around')

    ax = axs[1]                  # Commands to ax affect the bottom plot
    ax.set_yscale('log')
    ax.errorbar(x, data, yerr=[yerrL, yerrH], xerr=xerr,
            fmt='o', ecolor='g', capthick=2)          # Plot the data
    ax.plot(x,yfun,color='r')    # Plot the function
    ax.set_title('Giles playing around 2')

    fig.suptitle('File %s'%gFileName)
    fig.tight_layout()
    if saveFigs:
        fig.savefig("Fit.pdf", bbox_inches='tight')
        plt.close(fig)
    # plt.show()   # Do this in main to allow several panels of plots

## Class: MCMC output histograms
The next three functions are for making and displaying the histograms from the MCMC output, assuming four parameters are of interest.

In [0]:
class doOutputHistos4:    # There is always a 'self' parameter in functions in classes
    def __init__(self, Fit, Names):
        """
          Since we have four parameter variables in the fit, we reserve
          space for four 1-D projections of the probability distribution
          and six for the 2-D projections of the probability distributions.
          Use this class this way:
            oh = doOutputHistos4(gFit,Names)  # Once at start of program
            oh.fill(current)         # In the MCMC loop
            oh.plot()                # At the end
            plt.show()               # When all the plot panes have been set up.

          Fit is a numpy array.  It is usually the gFit variable from
          the setup part of the program at the top, but since this is a class, we could
          add several different sets of histograms if we want.
          Names is a python list with short names for each parameter for histogram titles
        """
        # The class has the following 'local variables' (initialise to None for now)
        self.histo1 = [None]*4   # list size 4 of numpy arrays for the 1-D histogram bins
        self.histo2 = [None]*6   # list size 6 of numpy 2-D arrays for the 2-D histograms
        self.idx = [None]*4      # list size 4 of integers giving row# in Fit of our 4 variables
        self.plotinfo = [None]*4 # list size 4 of python tuples (name, #bins, low edge, high edge)
        self.xvar = np.array([0,0,0,1,1,2])  # 2-D plot which variable on x axis
        self.yvar = np.array([1,2,3,2,3,3])  # 2-D plot which variable on y axis

        n2,n1 = Fit.shape      # n2 is #parameters (n1 is #columns in Fit)
        n0 = 0                 # Count the #parameters we find to histogram, must be =4
        for i in range(n2):
            if Fit[i,3] > 0.5: n0 += 1
        if n0 != 4:            # Warn and stop running
            print("In doOutputHisto4, found %d "%n0,
                  "variables enabled, must be 4.  Check gFit variable")
            sys.exit(1)

        # Step through the variable list in Fit, and make the memory areas for the plots
        # Usually if Fit just contains the four variables, so n0 = i
        n0 = 0
        nbin = np.zeros(4,dtype=np.int64)
        for i in range(n2):
            if Fit[i,3] > 0.5:
                nbin[n0] = int(Fit[i,0])
                self.idx[n0] = i
                self.plotinfo[n0] = (Names[i], nbin[n0], Fit[i,1], Fit[i,2])
                self.histo1[n0] = np.zeros(nbin[n0])
                n0 += 1
        self.histo2[0] = np.zeros((nbin[0],nbin[1]))   # 2-D correlation histos
        self.histo2[1] = np.zeros((nbin[0],nbin[2]))
        self.histo2[2] = np.zeros((nbin[0],nbin[3]))
        self.histo2[3] = np.zeros((nbin[1],nbin[2]))
        self.histo2[4] = np.zeros((nbin[1],nbin[3]))
        self.histo2[5] = np.zeros((nbin[2],nbin[3]))
        # The above 6 lines can in principle be shortened to
        # for n0 in range(6):
        #    self.histo2[n0][self.xvar[n0],self.yvar[n0]] +=1


    def fill(self,current):
        """
          Fill the step in the MCMC into the different projection histograms.
          We have already stored the info on the four parameters in the class variables
        """
        # Decide the bin for each of the four variables and fill the 1-D histogram
        bin = np.zeros(4,dtype=np.int64)
        for n0 in range(4):
            val = current[self.idx[n0]]
            (name, nbin, lowe, highe) = self.plotinfo[n0]
            b = int(nbin*(val-lowe)/(highe-lowe))
            if b < 0:      b = 0
            if b > nbin-1: b = nbin-1
            self.histo1[n0][b] += 1.
            bin[n0] = b    # Local array for filling 2-D histos
        self.histo2[0][bin[0],bin[1]] += 1   # 2-D correlation histos
        self.histo2[1][bin[0],bin[2]] += 1
        self.histo2[2][bin[0],bin[3]] += 1
        self.histo2[3][bin[1],bin[2]] += 1
        self.histo2[4][bin[1],bin[3]] += 1
        self.histo2[5][bin[2],bin[3]] += 1
        # The above 6 lines can in principle be shortened to
        # for n0 in range(6):
        #    self.histo2[n0][self.xvar[n0],self.yvar[n0]] +=1

    def plot(self, saveFigs=False):
        """
          Make a matplotlib plot with the four 1-D projection histograms and a
          matplotlib plot with the six 2-D projections on it.  Later we will
          try to find the 68% and 90% contours, but that is not done yet.
        """
        fig, axs = plt.subplots(nrows=2, ncols=2)  # Two plots on panel
        fig.patch.set_facecolor('w')
        axss = axs.flatten()

        for n0 in range(4):
            ax = axss[n0]
            (name, nbin, lowe, highe) = self.plotinfo[n0]
            x = np.linspace(lowe,highe,nbin)
            yerrV = np.sqrt(self.histo1[n0])
            ax.errorbar(x, self.histo1[n0], yerr=yerrV, fmt='o')  # Plot the 1D
            ax.set_title('Parameter %s'%name)
        fig.suptitle('1-D projection plots.  File %s' % gFileName)

        if saveFigs:
            fig.savefig("./1D_Plots.pdf", bbox_inches='tight')
            plt.close(fig)

        fig2, axs2 = plt.subplots(nrows=2, ncols=3)  # Six plots on panel
        fig2.patch.set_facecolor('w')
        axss = axs2.flatten()

        cm = ['RdBu_r', 'viridis']

        for n0 in range(6):
            ax = axss[n0]
            (namex,nbinx,lowex,highex) = self.plotinfo[self.xvar[n0]]
            (namey,nbiny,lowey,highey) = self.plotinfo[self.yvar[n0]]
            x = np.linspace(lowex,highex,nbinx) # pcolormesh needs N+1 bins
            y = np.linspace(lowey,highey,nbiny) # pcolormesh needs N+1 bins
            pcm = ax.pcolormesh(x[1:],y[1:], self.histo2[n0][1:-1,1:-1].T ,cmap=cm[0])
            ax.set_title("%s %s" % (namex,namey))
            fig2.colorbar(pcm,ax=ax)
        fig2.suptitle('2-D projection plots. File %s' % gFileName)

        if saveFigs:
            # Save as png as large number of bins causes problems for pdf
            fig2.savefig("./2D_Plots.png", bbox_inches='tight')
            plt.close(fig2)

## Class: history plots
The next three functions are for making and displaying the histograms from the MCMC history, assuming four parameters are of interest.

In [0]:
class doHistoryHistos4:    # There is always a 'self' parameter in functions in classes
    def __init__(self, Fit, Names):
        """
          Similar to the doOutputHistos class above, but for history plots
          The calling sequence is very similar:
            oh = doHistoryHistos4(gFit,Names)  # Once at start of program
            oh.fill(current,time)    # In the MCMC loop
            oh.plot()                # At the end
            plt.show()               # When all the plot panes have been set up.
        """
        # The class has the following 'local variables'
        self.histo1 = [None]*4  # list size 4 of numpy arrays for the 1-D histogram bins
        self.idx = [None]*4     # list size 4 of integers giving row# in Fit of our 4 variables
        self.names = [None]*4   # list size 4 of python strings with parameter name (to label plot)
        self.nbins = gHistoryMax/gHistoryStep + 1  # Number of bins in history

        n2,n1 = Fit.shape      # n2 is #parameters (n1 is #columns in Fit)
        n0 = 0                 # Count the #parameters we find to histogram, must be =4
        for i in range(n2):
            if Fit[i,3] > 0.5: n0 += 1
        if n0 != 4:            # Warn and stop running
            print("In doHistoryHisto4, found %d "%n0,
                  "variables enabled, must be 4.  Check gFit variable")
            sys.exit(1)

        # Check gHistoryMax and gHistoryStep are not crazy
        if gHistoryMax/gHistoryStep > 1000:
            print("In doHistoryHisto4, gHistoryStep and gHistoryMax give No. bins > 1000, stopping")
            sys.exit(1)

        # Step through the variable list in Fit, and make the memory areas for the plots
        # Usually if Fit just contains the four variables, so n0 = i
        n0 = 0
        for i in range(n2):
            if Fit[i,3] > 0.5:
                self.idx[n0] = i
                self.names[n0] = Names[i]
                self.histo1[n0] = np.zeros(self.nbins)
                n0 += 1

    def fill(self,current,time):
        """
          Fill the step into the history plots
        """
        if time < 0 or time > self.nbins-1:
            print("Warning in doHistoryPlots, time %d is out of range"%time)
            return
        for n0 in range(4):
            val = current[self.idx[n0]]
            self.histo1[n0][time] = val
            # print("hist fill debug: ",n0, val, self.idx[n0], time, self.histo1[n0][time])

    def plot(self, saveFigs):
        """
          Make a matplotlib plot with the four 1-D hitsory plots
        """
        fig, axs = plt.subplots(nrows=2, ncols=2, sharex=True)  # Two plots on panel
        fig.patch.set_facecolor('w')
        axss = axs.flatten()    # quash the 2-D array axs into a 4x1 1-D array

        for n0 in range(4):
            ax = axss[n0]
            x = np.arange(0, gHistoryMax+gHistoryStep, gHistoryStep)
            # print("History plot debug: ",n0,self.histo1[n0])
            ax.plot(x,self.histo1[n0],color='r')    # Plot the function
            ax.set_title('Parameter %s history' % self.names[n0])

        if saveFigs:
            fig.savefig("History.pdf", bbox_inches='tight')
            plt.close(fig)

# Simulation

## Prior
Return minus the log of the prior probability of obtaining the parameters given in current

The caller will then do np.exp(-ans) to get the probability.

In [0]:
def prior(current):
    return 0.       # Uniform prior for now: -log(1.)=0.

## Log-likelihood
Calculate the log-likelihood for the given parameters, under the probability model.

In [0]:
def prob(current):

    norm  = current[0]
    lam   = current[1]
    bkgd  = current[2]
    slope = current[3]

    # If any of the parameters are out of range, disqualify model (by making log-likelihood -> -inf)
    if (norm<0 or lam < 0 or bkgd < 0):
        return -2e-308
    else:
        # Do the log of poisson function here
        nbins, = gDataBins.shape # Should give size as a single number

        ans = 0.
        for x in range(35,nbins):
            mu = norm*np.exp(-x*lam/10000.) +bkgd
            ## print (np.exp(-x*lam/10000.))
            if gDataBins[x] != 0:
                ans1 = mu - gDataBins[x] + gDataBins[x]*np.log(gDataBins[x]/mu)
                ans += ans1
                ## print("x,ans ans1 mu data norm",x,ans,ans1,mu,gDataBins[x],norm,bkgd,lam)
            else:
                ans += mu

        # print("ans = ",ans)
        # This next line is the fictitious data, it runs fast to debug the MCMC software
        # ans = ((norm-560.)/4.)**2 + ((lam-280.)/20.)**2 + ((bkgd-99.)/3.)**2 + (slope-0.)**2

        # IMPORTANT: Return ans = -ln(prob), The caller will then do exp(-ans) to get the prob
        #            This allows the caller to subtract the logs rather than divide the probs to
        #            avoid rounding errors. 
        return ans                         # was return np.exp(-ans)

## Sample
Return a proposed new point in the parameter space. 

To respect the Metropolis-Hastings algorithm, this should be reversible (i.e. symmetric) and only depend on the current state.

We do what is normally done and move by a random amount. 

To speed up, rather than using a real Gaussian, we add 4 uniform random numbers together, which is a trick from very old slow computers.

In [0]:
def sample(current):
    n2,n1 = gFit.shape      # n2 is #parameters (n1 is #columns in Fit)
    rn = np.random.random((4,n2))    # Create 4 uniform random numbers per parameter
    next = np.zeros(n2)
    for i in range(n2):
        sig = gFit[i,4]
        r = (rn[i,0]+rn[i,1]+rn[i,2]+rn[i,3]-2.)/10.
        next[i] = current[i] + sig*r

    return next

## Metropolis-Hastings
Returns a sample of the posterior probability generated from the function prob() using the Metropolis-Hastings algorithm. 

The function sample() chooses the next test point according to the criteria of the Metrolplis-Hastings algorithm.
 

In [0]:
def metropHast(saveFigs=False):
    """
     The functions prob(), sample() and prior() are defined above.
     Returns a sample of the posterior probability generated from the
     function prob() using the Metropolis-Hastings algorithm to make
     a Markov Chain Monte Carlo.  The function sample() chooses the
     next test point according to the criteria needed by Metrolplis-Hastings.
    """

    # Return a numpy array where each row is an MC outcome.
    # There are 5 columns, 4 for the parameters and one (cnt) for
    # the number of times the MH algorithm sat in this place.

    # Question: How do we turn this array into a 1-D projection histogram
    # or a 2-D contour plot?

    n2,n1 = gFit.shape      # n2 is #parameters (n1 is #columns in Fit)
    current = np.zeros(n2)
    for i in range(n2):
        current[i] = gFit[i,5]   # Copy the starting values of parameters

    ntrials = 0
    nmoved = 0

    oh = doOutputHistos4(gFit,gNames)
    hh = doHistoryHistos4(gFit,gNames)

    cpost = prob(current) + prior(current)
    for a in range (gSampleMax):
        next = sample(current)
        nprior = prior(current)
        if nprior > -2.e+308:      # If new prior nprior is zero, skip calculating nprob
            nprob = prob(next)    # This is the bit that takes a long time
        else: nprob = -2.e+308
        npost = nprob + nprior

        # We have chosen our new point and calculated the probability
        # here, use the MH algorithm to choose whether to move
        logalpha = npost - cpost     # Log of acceptance ratio
        alpha = np.exp(-logalpha)    # Alpha [By doint the above manipulation with logs, we avoid very small numbers]
        u = np.random.random_sample()   # Pick a random between 0 and 1
        ## print "MHdebug: cpost=%7.5f npost=%7.5f alpha=%7.5f u=%7.5f"%(cpost,npost,alpha,u),        # This is python 2, in python 3 to skip newline, do print(3, end='')
        if u<alpha:          # Move to new sampling
            for i in range(n2): current[i] = next[i]
            cpost = npost
            nmoved += 1
            ##print(" accepted")
        ##else: print(" rejected")
        ntrials += 1

        if ntrials == gBurnIn:
            print("Last sample of burnin, now recording")
        if (ntrials % gSamplePrt) == 0:
            print("Processing %d of %d %d moved (fraction %f)"%(ntrials,gSampleMax,nmoved,(nmoved+0.001)/ntrials),current,alpha)
        if (ntrials % gHistoryStep == 0 and ntrials < gHistoryMax):
            hh.fill(current,ntrials/gHistoryStep)
        if (ntrials >= gBurnIn):
            oh.fill(current)

    # Finished all the samples, now do the plots
    print("Finished all %d samples.   Number moved was %d"%(ntrials,nmoved))
    oh.plot(saveFigs)
    hh.plot(saveFigs)

# Run

## Setup: parameters
Global variables begin with 'g'.

In [0]:
gNames = ["norm","lam","bkgd","slope"]    # Names of parameters in same order as in gFit


# Plot settings: one row for each parameter. Format is:
# [N. of bins in histogram, low edge of first bin, high edge of last bin, plot yes(1)/no(0) (max 4), step-size, start value]
gFit = np.array( [ [100.,  45.,   65., 1., 20.,  40.],     # N parameter (normalisation)
                   [100., 125.,  150., 1., 10., 100.],     # lam parameter (lifetime in bins)
                   [100.,  95.,  105., 1., 0.1,  0.3],     # background level parameter (per bin)
                   [100.,  -1.,   +1., 1.,  0.,   0.]      # slope on acceptance
                                                ], dtype=np.float64 )

gBurnIn      =  5000   # How many iterations of the MCMC do we throw away
gHistoryMax  = 50000   # For how long do we record the history in the history plots
gHistoryStep =   500   # We don't record every history sample, this is how many to step
gSampleMax   = 50000   # How long to run the MonteCarlo for
gSamplePrt   =   100   # How often should we print the status

gSaveFigs = False

## Running the simulation

In [0]:
# Mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
datafile = 'NP08-2019.dat'
databins = [] 

with open('/content/drive/My Drive/Colab Notebooks/NP04r/NP04 data/'+datafile, 'rb') as f:
    titleline = f.readline()
    headerline = f.readline()
    (cNChan,cRealT,cLiveT,cOChan,cCookie,cDetNo,cSegNo,StartD,StartT,Filename) = headerline.split();

# The cNChan etc variables are read as strings, convert to integer to use them
    NChan = int(cNChan)    # #channels in histogram (= #numbers to read from file still)
    RealT = int(cRealT)    # Time in seconds the data were collected for
    LiveT = int(cLiveT)    # Time the detector was accepting data (nearly same as RealT,
                           # but detector is dead for short time after receiving each pulse.
    print("NChan %d RealT %d LiveT %d"%(NChan,RealT,LiveT))

# Now read the channel contents, which is the rest of the file
    gDataBins = np.fromfile(f, sep=" ")
    # print(gDataBins)
    f.close()

# Do the Metroplois Hastings MCMC fit.  TODO = This should return the bet fit values
    metropHast(gSaveFigs)

# Construct a function with the same number of bins, not fit yet
    nbins, = gDataBins.shape # Should give size as a single number
    x = np.arange(nbins)    # 1D array 0,1,2,... nbins
    yfun = 2000.*np.exp(-x/50.)  # 1D array of an exponential function
    # print(x)
    # print(yfun)

    doDataPlot(nbins,gDataBins,yfun, gSaveFigs)
    if not gSaveFigs:
        plt.show()

NChan 512 RealT 4399663 LiveT 4399653


TypeError: ignored

# Output